load all the required libraries

In [2]:
import tensorflow as tf

# Set GPU memory growth to True
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)


2023-08-24 16:02:56.669473: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9498] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-08-24 16:02:56.669512: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-08-24 16:02:56.671264: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-24 16:02:56.768543: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-24 16:02:57.564256: W tensorflow/compiler/

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


2023-08-24 16:02:58.744484: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-24 16:02:58.828241: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-24 16:02:58.828386: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
# from proteinbert import load_pretrained_model
# from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

load the dataset

In [4]:
df = pd.read_csv('mutated.csv')

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load ProteinBERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert_bfd")
model = BertModel.from_pretrained("Rostlab/prot_bert_bfd")

# Protein sequence
protein_sequence = "MSIFLCFLLLLPLSLIFLKKLLPSKGKLPPGPKGLPIIGNLHQFGRFLHKSLHKISQEYGPVMLLHFGVVPVIIVSSKEGAEEVLKTHDLETCSRPKTVGSGLFTYNFKDIGFAPYGENWREMRKIAVSELFSQKKLKSFRYIREDESQLLVRKVSKSALETPTSSVNLRKVIFTFAASIICRLSFGQNFCDFVDMETVEELVLESETNLGSLAFADFLPAGWIIDRISGQHSTVMKAFSKLTNFFELVIDDHLKSGKIEDHSDIISVMLDMINKPTEVGSYKVTDDHLKGLMSDVFLAGVNAGSITMIWTMTELSRHPRVMRKLQEEIRAALGPNKEKITEEDLEKVEYLKMVIEEAFRLHPPAPLLLPRLTMSDINIQGYSIPKNTMIQINTYTIGRDPKNWTKPDEFIPERFVDNPIEYKGQHFELLPFGAGRRVCPGMATGITIVELGLLSLLYFFDWSLPNGMTTKDIDMEEDGAFVIAKKVSLELVPTLHRW"

# Encode the protein sequence
tokens = tokenizer.encode(protein_sequence, add_special_tokens=True)
input_ids = torch.tensor(tokens).unsqueeze(0)  # Add batch dimension

with torch.no_grad():
    outputs = model(input_ids)

encoded_sequence = outputs.last_hidden_state  # Use the last hidden state as the encoded representation

# Print the shape of the encoded sequence
print("Encoded Sequence Shape:", encoded_sequence.shape)


In [ ]:
encoded_sequence

In [ ]:
def encode_protein_sequences(protein_sequence):
    tokens = tokenizer.encode(protein_sequence, add_special_tokens=True)
    input_ids = torch.tensor(tokens).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        outputs = model(input_ids)

    encoded_sequence = outputs.last_hidden_state  # Use the last hidden state as the encoded representation
    return encoded_sequence

protein_sequences = df['mutated_sequence']
encoded_sequences = [encode_protein_sequences(seq) for seq in protein_sequences]


In [ ]:
encoded_sequences_array = np.array(encoded_sequences)

In [ ]:
encoded_sequences_array

In [ ]:
new = []
for i in encoded_sequences_array:
    new.append(np.array(i.flatten()))

In [ ]:
a= new[0]==new[1]

In [ ]:
for i in a:
    print(i)

In [ ]:
new_y = df['delta_G']
new_y = np.array(new_y)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(new, new_y, test_size=0.2)

# Initialize and train a Random Forest regressor
rf_model = RandomForestRegressor(n_estimators=10000)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared (R2) Score:", r2)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
X_train, X_test, y_train, y_test = train_test_split(new, new_y, test_size=0.1)

# Initialize and train various regression models
models = {
    "Random Forest": RandomForestRegressor(n_estimators=10),
    "Linear Regression": LinearRegression(),
    "Support Vector Regression": SVR()
}

results = {}  # To store the evaluation results for each model

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results[model_name] = {"MSE": mse, "R2": r2}

# Print the results
for model_name, metrics in results.items():
    print(model_name)
    print("Mean Squared Error:", metrics["MSE"])
    print("R-squared (R2) Score:", metrics["R2"])
    print()

In [ ]:
print(df)

In [ ]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import numpy as np
import pandas as pd

# Load the protein sequences
sequences = df['mutated_sequence']

combined_features = []

# Loop through each sequence
for sequence in sequences:
    # Calculate AAC
    protein_analysis = ProteinAnalysis(sequence)
    aac = list(protein_analysis.get_amino_acids_percent().values())

    # Calculate PSSM as simple counts
    pssm = [sequence.count(aa) for aa in "ARNDCQEGHILKMFPSTWYV"]

    # Combine AAC and PSSM features for each sequence
    combined = aac + pssm
    combined_features.append(combined)

# Convert features list to NumPy array
combined_array = np.array(combined_features)

# # Define column names
# aac_columns = list("ARNDCQEGHILKMFPSTWYV")
# pssm_columns = ["PSSM_" + aa for aa in "ARNDCQEGHILKMFPSTWYV"]
# combined_columns = aac_columns + pssm_columns

# # Create a DataFrame with combined features
# combined_df = pd.DataFrame(combined_array, columns=combined_columns)

# # Display the resulting DataFrame
# print(combined_df)


In [ ]:
combined_array[0].flatten().shape

In [ ]:
combined_df.columns

Encode the protein sequence

In [5]:
# Assuming 'df' is your DataFrame
protein_sequences = df['mutated_sequence']
delta_g_values = df['delta_G']

# Define the amino acids and their order
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

def one_hot_encode_sequence(sequence):
    # Create a dictionary to map each amino acid to an index
    aa_to_index = {aa: i for i, aa in enumerate(amino_acids)}
    
    # Initialize an array of zeros with the shape (sequence_length, number_of_amino_acids)
    sequence_length = len(sequence)
    one_hot_encoded = np.zeros((sequence_length, len(amino_acids)), dtype=int)
    
    # Set the appropriate index to 1 for each amino acid in the sequence
    for i, aa in enumerate(sequence):
        one_hot_encoded[i, aa_to_index[aa]] = 1
    
    return one_hot_encoded

# One-hot encode all the protein sequences in the DataFrame
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming you have a list of one-hot encoded sequences
encoded_sequences = [one_hot_encode_sequence(seq) for seq in protein_sequences]

#Find the maximum sequence length
max_sequence_length = max(len(seq) for seq in encoded_sequences)

# Pad sequences to the maximum length
padded_sequences = pad_sequences(encoded_sequences, maxlen=max_sequence_length, padding='post', truncating='post', dtype='float32')

# Convert the padded_sequences list to a numpy array
X = np.array(padded_sequences)

# Convert the delta_g_values to a numpy array
y = np.array(delta_g_values)


In [ ]:
import numpy as np

# Function to add random noise to one-hot encoded vectors
def add_noise(data, noise_level=0.05):
    noisy_data = data + noise_level * np.random.random(data.shape)
    noisy_data = np.clip(noisy_data, 0, 1)  # Clip values to [0, 1] range
    return noisy_data

# Augment the data with noise
augmented_data = np.array([add_noise(sample) for sample in X])

# Print some examples of the augmented data
# for i, augmented_sample in enumerate(augmented_data):
#     print(f"Augmented Sample {i+1}:\n{augmented_sample}")


In [ ]:
augmented_data

In [ ]:
# aug_500 = augmented_data[0:500]
# aug_1000 = augmented_data[500:1000]
# aug_1500 = augmented_data[1000:1500]
# aug_2000 = augmented_data[1500:2000]
# aug_2100 = augmented_data[2000:2100]


split the X,y value in train,test,validation

In [6]:
# Split the data into 70% training and 30% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3,random_state=42)

# # Split the remaining data (30%) into 10% validation and 20% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.67, random_state=42)

# # Print the shapes of the datasets
print("Training data shapes:", X_train.shape, y_train.shape)
print("Validation data shapes:", X_val.shape, y_val.shape)
print("Test data shapes:", X_test.shape, y_test.shape)


Training data shapes: (1469, 34350, 20) (1469,)
Validation data shapes: (207, 34350, 20) (207,)
Test data shapes: (423, 34350, 20) (423,)


In [ ]:
X_train

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.67,random_state=42)

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

# Set the input shape and number of classes (1 for regression)
input_shape = (X_train.shape[1:])
num_classes = 1

# Create the model architecture
def create_model():
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='linear'))
    
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
    return model

# Initialize lists to store R2 scores for each fold
r2_scores = []

# Create a KFold cross-validator
num_folds = 3
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Iterate through cross-validation folds
for train_idx, val_idx in kf.split(X_train):
    # Split the data into train and validation sets for this fold
    X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
    y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
    
    # Create a new instance of the model for each fold
    model = create_model()
    lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=5, verbose=1)
    # Train the model on the current fold's training data
    model.fit(X_fold_train, y_fold_train, epochs=100, batch_size=32, validation_data=(X_fold_val, y_fold_val), callbacks=[lr_scheduler])
    
    # Evaluate the model on the current fold's validation data
    y_fold_val_pred = model.predict(X_fold_val)
    r2 = r2_score(y_fold_val, y_fold_val_pred)
    r2_scores.append(r2)

# Calculate the mean and standard deviation of R2 scores


2023-08-24 16:03:19.241496: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-24 16:03:19.241655: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-24 16:03:19.241757: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/100


2023-08-24 16:03:23.473326: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-08-24 16:03:24.837837: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x10a201f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-24 16:03:24.837855: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-08-24 16:03:24.843845: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-24 16:03:24.912561: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


31/31 [==============================] - ETA: 0s - loss: 21.7190

2023-08-24 16:03:30.345247: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1346520000 exceeds 10% of free system memory.
2023-08-24 16:03:31.740457: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1346520000 exceeds 10% of free system memory.


31/31 [==============================] - 11s 256ms/step - loss: 21.7190 - val_loss: 27.9589 - lr: 0.0010
Epoch 2/100
31/31 [==============================] - 3s 112ms/step - loss: 15.1863 - val_loss: 13.3038 - lr: 0.0010
Epoch 3/100
31/31 [==============================] - 3s 110ms/step - loss: 13.5539 - val_loss: 13.2406 - lr: 0.0010
Epoch 4/100
31/31 [==============================] - 3s 110ms/step - loss: 11.9022 - val_loss: 10.0463 - lr: 0.0010
Epoch 5/100
31/31 [==============================] - 3s 111ms/step - loss: 9.8162 - val_loss: 10.2635 - lr: 0.0010
Epoch 6/100
31/31 [==============================] - 3s 110ms/step - loss: 8.4421 - val_loss: 8.4828 - lr: 0.0010
Epoch 7/100
31/31 [==============================] - 3s 111ms/step - loss: 8.2819 - val_loss: 8.7845 - lr: 0.0010
Epoch 8/100
31/31 [==============================] - 3s 110ms/step - loss: 7.1264 - val_loss: 8.4749 - lr: 0.0010
Epoch 9/100
31/31 [==============================] - 3s 111ms/step - loss: 6.2619 - val_lo

31/31 [==============================] - 3s 113ms/step - loss: 1.9523 - val_loss: 7.7264 - lr: 1.5625e-05
Epoch 64/100
31/31 [==============================] - ETA: 0s - loss: 1.9717
Epoch 64: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
31/31 [==============================] - 3s 112ms/step - loss: 1.9717 - val_loss: 7.7227 - lr: 1.5625e-05
Epoch 65/100
31/31 [==============================] - 3s 111ms/step - loss: 1.9300 - val_loss: 7.7341 - lr: 7.8125e-06
Epoch 66/100
31/31 [==============================] - 3s 111ms/step - loss: 2.0441 - val_loss: 7.7451 - lr: 7.8125e-06
Epoch 67/100
31/31 [==============================] - 3s 111ms/step - loss: 2.0690 - val_loss: 7.7430 - lr: 7.8125e-06
Epoch 68/100
31/31 [==============================] - 3s 111ms/step - loss: 2.0317 - val_loss: 7.7451 - lr: 7.8125e-06
Epoch 69/100
31/31 [==============================] - ETA: 0s - loss: 2.0708
Epoch 69: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
31/3

2023-08-24 16:09:13.879474: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1346520000 exceeds 10% of free system memory.


16/16 [==============================] - 1s 27ms/step


In [1]:
mean_r2 = np.mean(r2_scores)
std_r2 = np.std(r2_scores)

print(f"Mean R2 across {num_folds} folds: {mean_r2:.4f}")
print(f"Standard Deviation of R2: {std_r2:.4f}")


NameError: name 'np' is not defined

model creation and building

In [7]:
# Set the input shape and number of classes (1 for regression)
input_shape = (X_train.shape[1:])
num_classes = 1

# Create the model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='linear'))  # Using 'linear' activation for regression

# Compile the model with appropriate loss function and optimizer
model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))

# Apply learning rate scheduling
lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=5, verbose=1)

# Train the model using the training dataset and validate on the validation set
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), callbacks=[lr_scheduler])


2023-08-24 15:44:39.095438: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-24 15:44:39.095643: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-24 15:44:39.095784: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/100


2023-08-24 15:44:44.584997: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-08-24 15:44:45.949547: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f361841d4e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-24 15:44:45.949565: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-08-24 15:44:45.955360: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-24 15:44:46.028852: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


46/46 [==============================] - 11s 154ms/step - loss: 22.1182 - val_loss: 12.5629 - lr: 0.0010
Epoch 2/100
46/46 [==============================] - 4s 97ms/step - loss: 13.7499 - val_loss: 11.4033 - lr: 0.0010
Epoch 3/100
46/46 [==============================] - 5s 99ms/step - loss: 10.1893 - val_loss: 8.3092 - lr: 0.0010
Epoch 4/100
46/46 [==============================] - 5s 99ms/step - loss: 9.6446 - val_loss: 8.8041 - lr: 0.0010
Epoch 5/100
46/46 [==============================] - 4s 97ms/step - loss: 8.5449 - val_loss: 9.2243 - lr: 0.0010
Epoch 6/100
46/46 [==============================] - 4s 96ms/step - loss: 7.3802 - val_loss: 8.6016 - lr: 0.0010
Epoch 7/100
46/46 [==============================] - 4s 96ms/step - loss: 10.1476 - val_loss: 8.9323 - lr: 0.0010
Epoch 8/100
46/46 [==============================] - 4s 96ms/step - loss: 9.1451 - val_loss: 7.8513 - lr: 0.0010
Epoch 9/100
46/46 [==============================] - 4s 97ms/step - loss: 6.3679 - val_loss: 8.5504 

46/46 [==============================] - 4s 97ms/step - loss: 2.2267 - val_loss: 7.8950 - lr: 3.9063e-06
Epoch 62/100
46/46 [==============================] - ETA: 0s - loss: 2.5119
Epoch 62: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.
46/46 [==============================] - 4s 97ms/step - loss: 2.5119 - val_loss: 7.9280 - lr: 3.9063e-06
Epoch 63/100
46/46 [==============================] - 4s 97ms/step - loss: 2.4161 - val_loss: 7.9187 - lr: 1.9531e-06
Epoch 64/100
46/46 [==============================] - 4s 96ms/step - loss: 2.6452 - val_loss: 7.9081 - lr: 1.9531e-06
Epoch 65/100
46/46 [==============================] - 4s 97ms/step - loss: 2.6085 - val_loss: 7.9138 - lr: 1.9531e-06
Epoch 66/100
46/46 [==============================] - 4s 97ms/step - loss: 2.1673 - val_loss: 7.9120 - lr: 1.9531e-06
Epoch 67/100
46/46 [==============================] - ETA: 0s - loss: 2.4822
Epoch 67: ReduceLROnPlateau reducing learning rate to 9.765625463842298e-07.
46/46 [==

checking the mean_squared_error, r2_score

In [8]:
# Assuming you have predictions and true values as numpy arrays
prediction = model.predict(X_test)
y_true = np.array(y_test)  # True delta_G values for the test data
y_pred = prediction.reshape(-1)  # Reshape predictions to a 1D array if needed

# Calculate mean squared error (MSE)
mse = mean_squared_error(y_true, y_pred)

# Calculate coefficient of determination (R-squared score)
r2_score_value = r2_score(y_true, y_pred)

# Print the results
print("Mean Squared Error (MSE):", mse)
print("R-squared Score:", r2_score_value)


2023-08-24 15:52:23.827047: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1162404000 exceeds 10% of free system memory.
2023-08-24 15:52:24.703923: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1162404000 exceeds 10% of free system memory.


14/14 [==============================] - 1s 38ms/step
Mean Squared Error (MSE): 3.0743160628446993
R-squared Score: 0.7987661475012363


save the model

In [ ]:
model.save('change_data.h5')

for predicting Delta_G  of single sequence

In [ ]:
import numpy as np
from keras.models import load_model

# Protein sequence for prediction
protein_sequence = 'MIPLLGNSRETCGSDRGGFNKFGGPRDQGSRHDSEQDNSDNNTIFVQGLGENVTIESVADYFKQIGIIKTNKKTGQPMINLYTDRETGKLKGEATVSFDDPPSAKAAIDWFDGMYEKAGRGKEFSGNPIKVSFATRRADFNRGGGNGRGGRGRGGPMGRGGYGGGGSGGGGRGGFPSGGGGGGGQQRAGDWKCPNPTCENMNFSWRNECNQCKAPKPDGPGGGPGGSHMGGNYGDDRRGGRGGYDRGGYRGRGGDRGGFRGGRGGGDRGGFGPGKMDSRGEHRQDRRERPY'
# Load the model
model = load_model('/home/gdt-ws4/Desktop/prediction_all/change_data.h5')
# Define the amino acids and their order

amino_acids = "ACDEFGHIKLMNPQRSTVWY"

def one_hot_encode_sequence(sequence):
    # Create a dictionary to map each amino acid to an index
    aa_to_index = {aa: i for i, aa in enumerate(amino_acids)}
    
    # Initialize an array of zeros with the shape (sequence_length, number_of_amino_acids)
    sequence_length = len(sequence)
    one_hot_encoded = np.zeros((sequence_length, len(amino_acids)), dtype=int)
    
    # Set the appropriate index to 1 for each amino acid in the sequence
    for i, aa in enumerate(sequence):
        one_hot_encoded[i, aa_to_index[aa]] = 1
    
    return one_hot_encoded

# One-hot encode the protein sequence
encoded_sequence = one_hot_encode_sequence(protein_sequence)

# Pad the sequence to the desired length (34350)
desired_sequence_length = 34350
padded_encoded_sequence = np.pad(encoded_sequence, ((0, desired_sequence_length - len(encoded_sequence)), (0, 0)), 'constant')

# Reshape to a 3D array with shape (1, 34350, 20)
encoded_sequence_3D = padded_encoded_sequence[np.newaxis, :, :]

# Make predictions using the model
predicted_delta_g = model.predict(encoded_sequence_3D)

print("Predicted Delta_G value:", predicted_delta_g[0][0])


In [ ]:
Predicted Delta_G value: 6.130406
Predicted Delta_G value: 4.6328096

predicting all sequence in dataframe 

In [ ]:
# Assuming df is the DataFrame containing all the columns as described
X_test = df['Mutated Sequence']

# Define the maximum sequence length
max_sequence_length = 34350

# Assuming one_hot_encode_sequence function is defined as before

# Pad or truncate the sequences to the maximum length
encoded_sequences = [one_hot_encode_sequence(seq) for seq in X_test]
padded_sequences = pad_sequences(encoded_sequences, maxlen=max_sequence_length, padding='post', dtype='float32')

# Load the trained model from the file
model = load_model('change_data.h5')

# Make predictions using the loaded model
predicted_delta_g = model.predict(padded_sequences)

# Create a new DataFrame with all the columns and Predicted Delta_G values
df_predicted = pd.DataFrame({
    'Main': df['Main'],
    'Swiss-Prot': df['Swiss-Prot'],
    'AA': df['AA'],
    'Variant': df['Variant'],
    'FTId': df['FTId'],
    'dbSNP': df['dbSNP'],
    'Disease name': df['Disease name'],
    'new_variant': df['new_variant'],
    'sequences_data': df['sequences_data'],
    'Mutated Sequence': X_test,
    'Delta_G': predicted_delta_g.flatten()
})

# Display the DataFrame
print(df_predicted)
